# Interpolation

Interpolations are particularly useful when we repeatedly want to evaluate a function $f(x)$ that is computationally expensive to calculate. We then do the following

1. Calculate $f(x)$ values for a grid of $x$. This creates a "look-up" table.
2. In the expensive calculations, interpolate $f(x_i)$ by using the "look-up" table.

The notebook uses the package Interpolations (see https://github.com/JuliaMath/Interpolations.jl).

# Load Packages

In [2]:
using Interpolations

include("printmat.jl")   #just a function for prettier matrix printing

println4Ps (generic function with 1 method)

In [3]:
using Plots

backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))    
else    
    gr(size=(600,400))
end

Plots.GRBackend()

## Some Data to Be Interpolated

In [5]:
xGrid = collect(linspace(-pi,pi,101))            #equally spaced grid
yGrid = sin.(xGrid)                              #y values at xGrid 

plot1 = plot(xGrid,yGrid,color=:red,linewidth=2,legend=nothing)
title!("the sin function")
xlabel!("x")
ylabel!("y")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 -1.0 
 
 
 -0.5 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 the sin function 
 
 
 x 
 
 
 y 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 39.3701,188.189 44.8976,199.511 50.4252,210.788 55.9528,221.977 61.4803,233.031 67.0079,243.909 72.5354,254.567 78.063,264.963 83.5906,275.056 89.1181,284.807 
 94.6457,294.175 100.173,303.126 105.701,311.623 111.228,319.633 116.756,327.124 122.283,334.067 127.811,340.434 133.339,346.2 138.866,351.343 144.394,355.842 
 149.921,359.679 155.449,362.839 160.976,365.31 166.504,367.082 172.031,368.148 177.559,368.504 183.087,368.148 188.614,367.082 194.142,365.31 199.669,362.839 
 205.197,359.679 210.724,355.842 216.252,351.343 221.78,346.2 227.307,340.434 232.835,334.067 238.362,327.124 243.89,319.633 249.417,311.623 254.945,303.126 
 260.472,294.175 266,284.807 271.528,275.056 277.055,264.963 282.583,254.567 288.11,243.909 293.638,233.031 299.165,221.977 304.693,210.788 310.22,199.511 
 315.748,188.189 321.276,176.867 326.803,165.59 332.331,154.401 337.858,143.346 343.386,132.469 348.913,121.811 354.441,111.415 359.969,101.322 365.496,91.5714 
 371.024,82.2025 376.551,73.2519 382.079,64.7549 387.606,56.745 393.134,49.2539 398.661,42.3111 404.189,35.944 409.717,30.1778 415.244,25.0351 420.772,20.5364 
 426.299,16.6993 431.827,13.5389 437.354,11.0679 442.882,9.29585 448.409,8.22983 453.937,7.87402 459.465,8.22983 464.992,9.29585 470.52,11.0679 476.047,13.5389 
 481.575,16.6993 487.102,20.5364 492.63,25.0351 498.157,30.1778 503.685,35.944 509.213,42.3111 514.74,49.2539 520.268,56.745 525.795,64.7549 531.323,73.2519 
 536.85,82.2025 542.378,91.5714 547.906,101.322 553.433,111.415 558.961,121.811 564.488,132.469 570.016,143.346 575.543,154.401 581.071,165.59 586.598,176.867 
 592.126,188.189 
 "/>

## Preparing the Inter- and Extrapolation

(The cell below applies some of the available options.)

In [6]:
itp = interpolate(yGrid, BSpline(Quadratic(Flat())), OnGrid())
etp = extrapolate(itp, Flat())            #flat outside observed range of x

#to interpolate the y values at
x = [0.25;0.75]

2-element Array{Float64,1}:
 0.25
 0.75

## How to Find x (where we want to interpolate) in xGrid (pre-existing grid)

The function in the next cell helps making the translation from $x$ values to what the interpolation functions refer to (which are normalized versions of $x$). See the subsequent cell for how this works.

In [7]:
function CompressTo1NPs(x,MinX,MaxX,N)              #x2 = a + b*x, so that x2 is in [1,N]
  b = (N-1)/(MaxX-MinX)
  a = 1 - b*MinX
  x2 = a + b*x
  return x2
end

CompressTo1NPs (generic function with 1 method)

In [9]:
#first find at which indices of xGrid where x would be located 
x_clamped = CompressTo1NPs(x,minimum(xGrid),maximum(xGrid),length(xGrid))   

println("We want interpolation of f(x) at x = ")
printmat(x)

println("We can look at the values at x_clamped = ")
printmat(x_clamped)

println("Compare with closest available values in xGrid")
printmat(xGrid[round.(Int,x_clamped)])

We want interpolation of f(x) at x = 
     0.250
     0.750

We can look at the values at x_clamped = 
    54.979
    62.937

Compare with closest available values in xGrid
     0.251
     0.754



## Interpolate

In [10]:
#then interpolate as
y_interpolated = itp[x_clamped]

println("x and interpolated values")
printmat([x y_interpolated])

x and interpolated values
     0.250     0.247
     0.750     0.682



## Extrapolate

Extrapolation in- and outside xGrid (...but do really want to do that?) is similar.

In [11]:
x2             = [1.25;pi+0.1;pi+0.5]
x2_clamped     = CompressTo1NPs(x2,minimum(xGrid),maximum(xGrid),length(xGrid))
y_extrapolated = etp[x2_clamped]

println("x2 and extrapolated values")
printmat([x2 y_extrapolated])

x2 and extrapolated values
     1.250     0.949
     3.242     0.000
     3.642     0.000



## Looking at the Results

In [12]:
plot2 = plot(xGrid,yGrid,color=:red,linewidth=2,label="sin function")
scatter!(x,y_interpolated,color=:magenta,markersize=5,marker=:square,label="interpolated")
scatter!(x2,y_extrapolated,color=:blue,markersize=8,label="extrapolated")
title!("the sin function")
xlabel!("x")
ylabel!("y")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -2 
 
 
 0 
 
 
 2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 the sin function 
 
 
 x 
 
 
 y 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:2; stroke-opacity:1; fill:none" points="
 55.0141,188.189 59.8444,198.87 64.6747,209.509 69.505,220.064 74.3353,230.493 79.1656,240.755 83.9959,250.81 88.8262,260.618 93.6565,270.139 98.4868,279.338 
 103.317,288.176 108.147,296.62 112.978,304.636 117.808,312.193 122.638,319.26 127.469,325.81 132.299,331.816 137.129,337.256 141.959,342.108 146.79,346.352 
 151.62,349.972 156.45,352.953 161.281,355.284 166.111,356.956 170.941,357.962 175.772,358.297 180.602,357.962 185.432,356.956 190.262,355.284 195.093,352.953 
 199.923,349.972 204.753,346.352 209.584,342.108 214.414,337.256 219.244,331.816 224.074,325.81 228.905,319.26 233.735,312.193 238.565,304.636 243.396,296.62 
 248.226,288.176 253.056,279.338 257.887,270.139 262.717,260.618 267.547,250.81 272.377,240.755 277.208,230.493 282.038,220.064 286.868,209.509 291.699,198.87 
 296.529,188.189 301.359,177.508 306.189,166.869 311.02,156.314 315.85,145.885 320.68,135.623 325.511,125.568 330.341,115.76 335.171,106.239 340.002,97.0403 
 344.832,88.2017 349.662,79.7578 354.492,71.7417 359.323,64.1853 364.153,57.1182 368.983,50.5683 373.814,44.5617 378.644,39.1218 383.474,34.2702 388.305,30.0261 
 393.135,26.4062 397.965,23.4248 402.795,21.0936 407.626,19.4219 412.456,18.4162 417.286,18.0805 422.117,18.4162 426.947,19.4219 431.777,21.0936 436.607,23.4248 
 441.438,26.4062 446.268,30.0261 451.098,34.2702 455.929,39.1218 460.759,44.5617 465.589,50.5683 470.42,57.1182 475.25,64.1853 480.08,71.7417 484.91,79.7578 
 489.741,88.2017 494.571,97.0403 499.401,106.239 504.232,115.76 509.062,125.568 513.892,135.623 518.722,145.885 523.553,156.314 528.383,166.869 533.213,177.508 
 538.044,188.189 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 sin function 
 
 
 
 
 interpolated 
 
 
 
 
 extrapolated